In [6]:
pip install scikit-learn imbalanced-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import pandas as pd
from imblearn.over_sampling import ADASYN


In [8]:
# Load data from CSV file
data = pd.read_csv('/content/df_cleaned_final.csv')

In [4]:
data.head()

,age,job,marital,education,housing,loan,contact,month,day_of_week,duration,campaign,previous,poutcome,y,PC1,PC2
0,56,housemaid,married,basic.4y,no,no,telephone,may,mon,261,1,0,nonexistent,0,-1.35089,0.681003
1,57,services,married,high.school,no,no,telephone,may,mon,149,1,0,nonexistent,0,-1.35089,0.681003
2,37,services,married,high.school,yes,no,telephone,may,mon,226,1,0,nonexistent,0,-1.35089,0.681003
3,40,admin.,married,basic.6y,no,no,telephone,may,mon,151,1,0,nonexistent,0,-1.35089,0.681003
4,56,services,married,high.school,no,yes,telephone,may,mon,307,1,0,nonexistent,0,-1.35089,0.681003


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37089 entries, 0 to 37088
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          37089 non-null  int64  
 1   job          37089 non-null  object 
 2   marital      37089 non-null  object 
 3   education    37089 non-null  object 
 4   housing      37089 non-null  object 
 5   loan         37089 non-null  object 
 6   contact      37089 non-null  object 
 7   month        37089 non-null  object 
 8   day_of_week  37089 non-null  object 
 9   duration     37089 non-null  int64  
 10  campaign     37089 non-null  int64  
 11  previous     37089 non-null  int64  
 12  poutcome     37089 non-null  object 
 13  y            37089 non-null  int64  
 14  PC1          37089 non-null  float64
 15  PC2          37089 non-null  float64
dtypes: float64(2), int64(5), object(9)
memory usage: 4.5+ MB


In [9]:
print ("Number of unique values for each variable")
print("="*20)

for feature in data.columns.values:
    print(f"{feature} ({data[feature].dtypes}): {data[feature].nunique()}")
    print("-"*20)

Number of unique values for each variable
age (int64): 78
--------------------
job (object): 11
--------------------
marital (object): 3
--------------------
education (object): 7
--------------------
housing (object): 2
--------------------
loan (object): 2
--------------------
contact (object): 2
--------------------
month (object): 10
--------------------
day_of_week (object): 5
--------------------
duration (int64): 753
--------------------
campaign (int64): 7
--------------------
previous (int64): 8
--------------------
poutcome (object): 3
--------------------
y (int64): 2
--------------------
PC1 (float64): 373
--------------------
PC2 (float64): 373
--------------------


In [10]:
# Separate features and labels
features = data.drop('poutcome', axis=1)
labels = data['poutcome']

In [11]:
# Define your features and labels explicitly
categorical_features = ['job',	'marital',	'education',	'housing',	'loan',	'contact',	'month',	'day_of_week', ]  # Specify your categorical feature column names
numeric_features = ['age', 'y']
#numeric_features = ['age	', 'duration', 'campaign', 'previous', 'y	', 'PC1', 'PC2']  # Specify your numeric feature column names
labels = data['poutcome']  # Specify your label column name


In [12]:
# Perform one-hot encoding for categorical features
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
encoded_categorical_features = pd.DataFrame(encoder.fit_transform(data[categorical_features]))

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [13]:
# Convert feature names to string type
encoded_categorical_features.columns = encoded_categorical_features.columns.astype(str)


In [14]:
# Combine encoded categorical features and numeric features
features = pd.concat([encoded_categorical_features, data[numeric_features]], axis=1)


In [15]:
# Convert feature names to string type
features.columns = features.columns.astype(str)


In [16]:
# Create an instance of ADASYN
adasyn = ADASYN()

In [17]:
# Perform ADASYN oversampling
oversampled_features, oversampled_labels = adasyn.fit_resample(features, labels)


In [18]:
# Print the class distribution before and after ADASYN
print("Class distribution before ADASYN:")
print(labels.value_counts())
print("\nClass distribution after ADASYN:")
print(oversampled_labels.value_counts())

Class distribution before ADASYN:
nonexistent    31803
failure         4008
success         1278
Name: poutcome, dtype: int64

Class distribution after ADASYN:
nonexistent    31803
success        31545
failure        31390
Name: poutcome, dtype: int64


In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

In [20]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [21]:
# Define the model to be used (e.g., Random Forest Classifier)
model = RandomForestClassifier()


In [22]:
# Define the parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'class_weight': ['balanced', None]
}

In [26]:
# Perform GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(model, param_grid, scoring='f1', cv=5)
grid_search.fit(X_train, y_train)


Streaming output truncated to the last 5000 lines.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1146, in f1_score
    return fbeta_score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1287, in fbeta_score
    _, _, f, _ = precision_recall_fscore_support(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1573, in precision_recall_fscore_support
    labels = _check_set_wise_labels(y_tru

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'class_weight': ['balanced', None],
                         'max_depth': [None, 5, 10],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='f1')

In [27]:
# Get the best model and parameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

In [28]:
# Evaluate the best model on the test set
y_pred = best_model.predict(X_test)
report = classification_report(y_test, y_pred)


In [29]:
# Print the best parameters and model evaluation report
print("Best Parameters:", best_params)
print("\nClassification Report:")
print(report)

Best Parameters: {'class_weight': 'balanced', 'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}

Classification Report:
              precision    recall  f1-score   support

     failure       0.27      0.20      0.23       793
 nonexistent       0.88      0.92      0.90      6377
     success       0.24      0.14      0.18       248

    accuracy                           0.82      7418
   macro avg       0.46      0.42      0.44      7418
weighted avg       0.79      0.82      0.81      7418



In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier


In [31]:
# Define the classifiers to be used
classifiers = [
    ('Logistic Regression', LogisticRegression()),
    ('Decision Tree', DecisionTreeClassifier()),
    ('Random Forest', RandomForestClassifier()),
    ('Gradient Boosting', GradientBoostingClassifier()),
    ('K-Nearest Neighbors', KNeighborsClassifier()),
    ('XGBoost', XGBClassifier())
]

In [32]:
# Define the parameter grids for hyperparameter tuning
param_grids = [
    {'penalty': ['l2'], 'C': [0.1, 1, 10]},
    {'max_depth': [None, 5, 10], 'min_samples_split': [2, 5, 10]},
    {'n_estimators': [100, 200, 300], 'max_depth': [None, 5, 10], 'min_samples_split': [2, 5, 10]},
    {'n_estimators': [100, 200, 300], 'learning_rate': [0.1, 0.01, 0.001]},
    {'n_neighbors': [3, 5, 7]},
    {'max_depth': [3, 5, 7], 'learning_rate': [0.1, 0.01, 0.001]}
]


In [33]:
# Define the evaluation metrics
metrics = {
    'accuracy': accuracy_score,
    'precision': lambda y_true, y_pred: precision_score(y_true, y_pred, average='weighted', zero_division=0),
    'recall': lambda y_true, y_pred: recall_score(y_true, y_pred, average='weighted', zero_division=0),
    'f1': lambda y_true, y_pred: f1_score(y_true, y_pred, average='weighted', zero_division=0),
    'roc_auc': lambda y_true, y_pred: roc_auc_score(y_true, y_pred, average='weighted')
}


In [34]:
#ne code with 
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Encode class labels to numeric values
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Loop over classifiers, perform GridSearchCV, and evaluate on test set
for clf_name, clf in classifiers:
    param_grid = param_grids[classifiers.index((clf_name, clf))]
    grid_search = GridSearchCV(clf, param_grid, scoring='f1', cv=5)
    grid_search.fit(X_train, y_train_encoded)

    best_model = grid_search.best_estimator_
    y_pred_encoded = best_model.predict(X_test)

    # Decode predicted labels back to original class labels
    y_pred = label_encoder.inverse_transform(y_pred_encoded)

    print("Classifier:", clf_name)
    print("Best Parameters:", grid_search.best_params_)

    for metric_name, metric_func in metrics.items():
        try:
            if metric_name == 'roc_auc':
                # Convert class labels to binary values for ROC AUC calculation
                y_test_binary = np.where(y_test == 'success', 1, 0)
                y_pred_binary = np.where(y_pred == 'success', 1, 0)
                score = metric_func(y_test_binary, y_pred_binary)
            else:
                score = metric_func(y_test, y_pred)
            print(f"{metric_name.capitalize()}: {score:.4f}")
        except ValueError:
            print(f"Error: Unable to calculate {metric_name} due to non-numeric values in y_test or y_pred")

    print("-----------")


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 234, in __cal

Classifier: Logistic Regression
Best Parameters: {'C': 0.1, 'penalty': 'l2'}
Accuracy: 0.8582
Precision: 0.8079
Recall: 0.8582
F1: 0.8007
Roc_auc: 0.5547
-----------


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1146, in f1_score
    return fbeta_score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1287, in fbeta_score
    _, _, f, _ = precision_recall_fscore_support(
  File "/usr/local/l

Classifier: Decision Tree
Best Parameters: {'max_depth': None, 'min_samples_split': 2}
Accuracy: 0.7884
Precision: 0.7983
Recall: 0.7884
F1: 0.7931
Roc_auc: 0.5896
-----------


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1146, in f1_score
    return fbeta_score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1287, in fbeta_score
    _, _, f, _ = precision_recall_fscore_support(
  File "/usr/local/l

...

Classifier: Gradient Boosting
Best Parameters: {'learning_rate': 0.1, 'n_estimators': 100}
Accuracy: 0.8585
Precision: 0.7820
Recall: 0.8585
F1: 0.8010
Roc_auc: 0.5456
-----------


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1146, in f1_score
    return fbeta_score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1287, in fbeta_score
    _, _, f, _ = precision_recall_fscore_support(
  File "/usr/local/l

Classifier: K-Nearest Neighbors
Best Parameters: {'n_neighbors': 3}
Accuracy: 0.8305
Precision: 0.7976
Recall: 0.8305
F1: 0.8096
Roc_auc: 0.5358
-----------


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1146, in f1_score
    return fbeta_score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1287, in fbeta_score
    _, _, f, _ = precision_recall_fscore_support(
  File "/usr/local/l

Classifier: XGBoost
Best Parameters: {'learning_rate': 0.1, 'max_depth': 3}
Accuracy: 0.8591
Precision: 0.7898
Recall: 0.8591
F1: 0.7992
Roc_auc: 0.5343
-----------
